In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb

from sklearn.preprocessing import LabelEncoder
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import auc

In [2]:
train_test = pd.read_csv("../bosch/train_test.csv")

In [3]:
train_test.head()

,Unnamed: 0,index,Id,Response,StartTime,0_¯\_(ツ)_/¯_1,0_¯\_(ツ)_/¯_2,0_¯\_(ツ)_/¯_3,0_¯\_(ツ)_/¯_4
0,2083747,2083747,1800056,NaN,-1,2,-1,9999999,-1
1,2083748,2083748,1800057,NaN,-1,1,-2,1,-2
2,2083749,2083749,1800059,NaN,-1,2,-6,2,-6
3,2083750,2083750,1800065,NaN,-1,6,-1,6,-1
4,2083751,2083751,1800066,NaN,-1,1,-5,1,-5


In [4]:
train = pd.read_csv("../bosch/train_numeric.csv")

In [5]:
trainp = pd.merge(train, train_test, on='Id', how='left')

In [6]:
del train
import gc
gc.collect()

30

In [8]:
import xgboost as xgb
Y=trainp['Response_x']
listp=[x for x in trainp.columns if x not in ["Response_x","Response_y"]]
len(listp)
X=trainp[listp]

In [10]:
import time
startt = time.time()
gbm = xgb.XGBClassifier(max_depth=10, n_estimators=200, learning_rate=0.1).fit(X, Y)
endt = time.time()
print((endt-startt)/60)

34.5558894316


### grid search

In [9]:
gbm_params = {
    'learning_rate': [0.05, 0.1],
    'n_estimators': [3],
    'max_depth': [2, 3],
}
cv = StratifiedKFold(Y)
cv

sklearn.cross_validation.StratifiedKFold(labels=[0 0 0 ..., 0 0 0], n_folds=3, shuffle=False, random_state=None)

In [11]:
gbm = xgb.XGBClassifier()
grid = GridSearchCV(gbm, gbm_params,scoring='roc_auc',cv=cv,verbose=10,n_jobs=-1)
grid.fit(X, Y)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] n_estimators=3, learning_rate=0.05, max_depth=2 .................
[CV] n_estimators=3, learning_rate=0.05, max_depth=2 .................
[CV] n_estimators=3, learning_rate=0.05, max_depth=2 .................
[CV]  n_estimators=3, learning_rate=0.05, max_depth=2, score=0.862758 -  55.1s


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.4min


[CV] n_estimators=3, learning_rate=0.05, max_depth=3 .................
[CV] n_estimators=3, learning_rate=0.05, max_depth=3 .................
[CV]  n_estimators=3, learning_rate=0.05, max_depth=2, score=0.859008 - 1.1min
[CV]  n_estimators=3, learning_rate=0.05, max_depth=2, score=0.303701 -  59.7s
[CV] n_estimators=3, learning_rate=0.05, max_depth=3 .................
[CV] n_estimators=3, learning_rate=0.1, max_depth=2 ..................
[CV]  n_estimators=3, learning_rate=0.05, max_depth=3, score=0.862833 - 1.2min
[CV] n_estimators=3, learning_rate=0.1, max_depth=2 ..................
[CV]  n_estimators=3, learning_rate=0.05, max_depth=3, score=0.859664 - 1.3min
[CV] n_estimators=3, learning_rate=0.1, max_depth=2 ..................
[CV]  n_estimators=3, learning_rate=0.05, max_depth=3, score=0.398787 - 1.3min
[CV] n_estimators=3, learning_rate=0.1, max_depth=3 ..................
[CV]  n_estimators=3, learning_rate=0.1, max_depth=2, score=0.862693 - 1.2min
[CV] n_estimators=3, learning_

[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  5.5min finished


GridSearchCV(cv=sklearn.cross_validation.StratifiedKFold(labels=[0 0 ..., 0 0], n_folds=3, shuffle=False, random_state=None),
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [3], 'learning_rate': [0.05, 0.1], 'max_depth': [2, 3]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=10)

In [12]:
grid.best_params_

{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 3}

In [ ]:
testX = X
predictions = grid.best_estimator_.predict(test_X)

In [11]:
del trainp
del Y
del X
import gc
gc.collect()

292

In [14]:
test = pd.read_csv("../bosch/test_numeric.csv")

In [17]:
testp = pd.merge(test, train_test, on='Id', how='left')

In [18]:
testX = testp[listp]

In [19]:
import time
import math
startt = time.time()
predictions = gbm.predict(testX)
endt = time.time()
print((endt-startt)/60)

0.964323286215


In [20]:
resultSubmission = pd.DataFrame({'Id':test['Id'],'Response':predictions})

In [21]:
resultSubmission.to_csv('submitResponse.csv',index=False)

In [22]:
resultSubmission.shape

(1183748, 2)

In [ ]:
skf = cross_validation.StratifiedKFold(y, n_folds=2)